In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torchvision.transforms.functional as F
from scipy import ndimage
from scipy.ndimage import gaussian_filter
from numpy import save

# from resnet_feature_extraction import Img2Vec
import torchvision.transforms as transforms
import torch.nn as nn
from model import Model

# Download CIFAR 10 data form torchvision

In [2]:

train_dataset = torchvision.datasets.CIFAR10('~/datasets/cifar', train=True, download=True)
test_dataset = torchvision.datasets.CIFAR10('~/datasets/cifar', train=False, download=True)


Files already downloaded and verified
Files already downloaded and verified


In [3]:
train_dataset.data.shape

(50000, 32, 32, 3)

In [4]:
a = torch.tensor(train_dataset.data)
a = torch.einsum('nhwc->nchw', a)
a.shape

torch.Size([50000, 3, 32, 32])

# Useful functions 

In [5]:
def apply_contrast(images_data, contrast_factor):
    to_tensor = transforms.ToTensor()   
    images_list = [to_tensor(im) for im in images_data]
    images = torch.stack(images_list)
    contrasted_images = F.adjust_contrast(images, contrast_factor)
    contrasted_images = np.array(np.stack([transforms.ToPILImage()(im) for im in contrasted_images]))
    return contrasted_images

def apply_rotation(images_data, angle=30):
    rotated_images = []
    for img in images_data:
        rotated_image = ndimage.rotate(img, angle, reshape=False)
        rotated_images.append(rotated_image)
    rotated_images = np.array(rotated_images)
    return rotated_images

def blur_images(images_data, sigma=1):
    blurred_images = []
    for img in images_data:
        blurred_image = gaussian_filter(img, sigma)
        blurred_images.append(blurred_image)
    blurred_images = np.array(blurred_images)
    return blurred_images


def apply_saturation(images_data, sat_factor):
    to_tensor = transforms.ToTensor()   
    images_list = [to_tensor(im) for im in images_data]
    images = torch.stack(images_list)
    saturated_images = F.adjust_saturation(images, sat_factor)
    saturated_images = np.array(np.stack([transforms.ToPILImage()(im) for im in saturated_images]))
    return saturated_images


#        scaler = transforms.Resize((224, 224))
#        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                               std=[0.229, 0.224, 0.225])
        
        
#         to_tensor = transforms.ToTensor()

# # Function to extract image features using resnet    
# def get_features(images, batch_size):
#     Z_list = []
#     # img2vec = Img2Vec(model="resnet50")
#     img2vec = Img2Vec()
#     for first in range(0, len(images), batch_size):
#         images_subset = images[first:first+batch_size]
#         Z_subset = img2vec.get_vec(images_subset)
#         Z_list.append(Z_subset)
#     Z = np.vstack(Z_list)
#     print("Z features shape", Z.shape)
#     return Z



In [6]:
# import torch
# import torch.nn as nn
import torch.nn.functional as Fu
# from torchvision.models.resnet import resnet50


# class Model(nn.Module):
#     def __init__(self, feature_dim=128):
#         super(Model, self).__init__()

#         self.f = []
#         for name, module in resnet50().named_children():
#             if name == 'conv1':
#                 module = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
#             if not isinstance(module, nn.Linear) and not isinstance(module, nn.MaxPool2d):
#                 self.f.append(module)
#         # encoder
#         self.f = nn.Sequential(*self.f)
#         # projection head
#         self.g = nn.Sequential(nn.Linear(2048, 512, bias=False), nn.BatchNorm1d(512),
#                                nn.ReLU(inplace=True), nn.Linear(512, feature_dim, bias=True))

#     def forward(self, x):
#         x = self.f(x)
#         feature = torch.flatten(x, start_dim=1)
#         out = self.g(feature)
#         return Fu.normalize(feature, dim=-1)#, F.normalize(out, dim=-1)


class Net(nn.Module):
    def __init__(self, num_class, pretrained_path):
        super(Net, self).__init__()

        # encoder
        model = Model()#.cuda()
        model = nn.DataParallel(model)
        model.load_state_dict(torch.load(pretrained_path))

        self.f = model.module.f
        # classifier
        self.fc = nn.Linear(2048, num_class, bias=True)

    def forward(self, x):
        x = self.f(x)
        feature = torch.flatten(x, start_dim=1)
        out = self.fc(feature)
        return Fu.normalize(feature, dim=-1)#feature #out


def get_features(model,images):
    x = torch.tensor(images)
    x = torch.einsum('nhwc->nchw', x)   
    extracted_features = None
    with torch.no_grad():
        extracted_features = model(x.float())
    extracted_features = extracted_features.numpy()
    print("z features shape", extracted_features.shape)
    return extracted_features  

# Transform images and extract their features

In [7]:
# Transform images 

# Transformation parameters
angle = 15 # for rotation
contrast_factor = 0.3 # for contrast
sigma = 0.5 # for blur
saturation_factor = 5 # for saturation

batch_size = 64

train_labels = np.array(train_dataset.targets)
test_labels = np.array(test_dataset.targets)

X_train_og = train_dataset.data
X_train_rotated = apply_rotation(train_dataset.data, angle)
X_train_contrasted = apply_contrast(train_dataset.data, contrast_factor) 
X_train_blurred = blur_images(train_dataset.data, sigma)
X_train_saturated = apply_saturation(train_dataset.data, saturation_factor)


X_test_og = test_dataset.data
X_test_rotated = apply_rotation(test_dataset.data, angle)
X_test_contrasted = apply_contrast(test_dataset.data, contrast_factor) 
X_test_blurred = blur_images(test_dataset.data, sigma)
X_test_saturated = apply_saturation(test_dataset.data, saturation_factor)


In [8]:
model_path='ipirm/model_400.pth' 
num_classes = 10
ip_irm_model = Net(num_class=num_classes, pretrained_path=model_path)#.cuda()

# # encoder
# ip_irm_model = Model()#.cuda()
# # model = nn.DataParallel(model)
# ip_irm_model.load_state_dict(torch.load(model_path))
# ip_irm_model = nn.DataParallel(ip_irm_model)
ip_irm_model.eval()

# Extract image features 
print("### Extracting features using a pre-trained resnet 18 ###")
Z_train_og = get_features(ip_irm_model, X_train_og)
Z_train_rotated = get_features(ip_irm_model, X_train_rotated)
Z_train_contrasted = get_features(ip_irm_model, X_train_contrasted)
Z_train_blurred = get_features(ip_irm_model, X_train_blurred)
Z_train_saturated = get_features(ip_irm_model, X_train_saturated)


Z_test_og = get_features(ip_irm_model, X_test_og)
Z_test_rotated = get_features(ip_irm_model, X_test_rotated)
Z_test_contrasted = get_features(ip_irm_model, X_test_contrasted)
Z_test_blurred = get_features(ip_irm_model, X_test_blurred)
Z_test_saturated = get_features(ip_irm_model, X_test_saturated)


### Extracting features using a pre-trained resnet 18 ###
z features shape (50000, 2048)
z features shape (50000, 2048)
z features shape (50000, 2048)
z features shape (50000, 2048)
z features shape (50000, 2048)
z features shape (10000, 2048)
z features shape (10000, 2048)
z features shape (10000, 2048)
z features shape (10000, 2048)
z features shape (10000, 2048)


In [9]:
# Save extracted features and image labels as numpy files

save('../data_ip_irm_new/Z_train_og_cifar10_ip_irm.npy', Z_train_og)
save('../data_ip_irm_new/Z_train_rotated_cifar10_ip_irm.npy', Z_train_rotated)
save('../data_ip_irm_new/Z_train_contrasted_cifar10_ip_irm.npy', Z_train_contrasted)
save('../data_ip_irm_new/Z_train_blurred_cifar10_ip_irm.npy', Z_train_blurred)
save('../data_ip_irm_new/Z_train_saturated_cifar10_ip_irm.npy', Z_train_saturated)

save('../data_ip_irm_new/Z_test_og_cifar10_ip_irm.npy', Z_test_og)
save('../data_ip_irm_new/Z_test_rotated_cifar10_ip_irm.npy', Z_test_rotated)
save('../data_ip_irm_new/Z_test_contrasted_cifar10_ip_irm.npy', Z_test_contrasted)
save('../data_ip_irm_new/Z_test_blurred_cifar10_ip_irm.npy', Z_test_blurred)
save('../data_ip_irm_new/Z_test_saturated_cifar10_ip_irm.npy', Z_test_saturated)

save('../data_ip_irm_new/train_labels_cifar10.npy', train_labels)
save('../data_ip_irm_new/test_labels_cifar10.npy', test_labels)